# 实验六 决策树分类器
## 2210529 罗瑞

## 基本要求

基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；
## 中级要求

对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
对测试集Watermelon-test2进行预测，输出分类精度；
## 高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

## ***基本要求实现***

### **加载数据**
使用load_txt方法从train1.scv 中读入数据，将离散的特征值（如“青绿”“蜷缩”等）转换为索引表示，将会返回一个NumPy数组，数组的每一行表示一个样本。
输出格式每一行的格式是 [编号, 色泽索引, 根蒂索引, 敲声索引, 纹理索引, 标签索引]。

In [35]:
import numpy as np
import codecs
feature_dict = {"色泽": ["青绿", "乌黑", "浅白"],
                "根蒂": ["蜷缩", "稍蜷", "硬挺"],
                "敲声": ["浊响", "沉闷", "清脆"],
                "纹理": ["清晰", "稍糊", "模糊"]
                }
lable_list = ["否", "是"]
feature_list = ["色泽", "根蒂", "敲声", "纹理"]

def load_txt(path):
    ans = []
    with codecs.open(path, "r", "GBK") as f:
        line = f.readline()  # 读取表头，跳过
        line = f.readline()  # 读取数据第一行
        while line:
            d = line.rstrip("\r\n").split(',')  # 按逗号分割每行数据
            re = []
            re.append(int(d[0]))  # 第0列是编号，作为辅助信息
            re.append(feature_dict.get("色泽").index(d[1]))  # 将“色泽”属性映射为索引
            re.append(feature_dict.get("根蒂").index(d[2]))  # 将“根蒂”属性映射为索引
            re.append(feature_dict.get("敲声").index(d[3]))  # 将“敲声”属性映射为索引
            re.append(feature_dict.get("纹理").index(d[4]))  # 将“纹理”属性映射为索引
            re.append(lable_list.index(d[-1]))  # 最后一列是标签（“是”或“否”），映射为索引
            ans.append(np.array(re))  # 存储为NumPy数组
            line = f.readline()  # 继续读取下一行
    return np.array(ans)


### **定义信息熵与信息增益的计算**

分别使用ent和gain函数。

#### 信息熵的计算:ent函数
$$H(D)=-\sum_{k=1}^Kp_k\log_2p_k$$
输入：类别标签数组D（如[0, 1, 1, 0]）。
输出：数据集的熵，表示类别分布的不确定性。


#### 信息增益的计算gain函数
$$Gain(D,A)=H(D)-\sum_{v\in A}\frac{|D_v|}{|D|}H(D_v)$$
输入：样本集X，类别标签Y，属性索引attr。
输出：在该属性上的信息增益。

In [4]:
def ent(D):
    s = 0
    for k in set(D):
        p_k = np.sum(np.where(D == k, 1, 0)) / np.shape(D)[0]  # 每个类别k的概率
        if p_k == 0:
            continue  # 概率为0时，熵贡献为0
        s += p_k * np.log2(p_k)  # 计算熵公式
    return -s

def gain(X, Y, attr):
    x_attr_col = X[:, attr]  # 获取当前属性列
    ent_Dv = []  # 子集的熵
    weight_Dv = []  # 子集的权重
    for x_v in set(x_attr_col):  # 遍历属性的所有取值
        index_x_equal_v = np.where(x_attr_col == x_v)
        y_x_equal_v = Y[index_x_equal_v]  # 对应的子集标签
        ent_Dv.append(ent(y_x_equal_v))  # 子集的熵
        weight_Dv.append(np.shape(y_x_equal_v)[0] / np.shape(Y)[0])  # 子集权重
    return ent(Y) - np.sum(np.array(ent_Dv) * np.array(weight_Dv))  # 信息增益公式


### **定义决策树节点：Node**

如果attr == -1，表示这是叶节点，label存储类别。
如果label == π，表示非叶节点，attr存储划分属性。
children存储当前节点的所有子节点。


In [13]:
class Node:
    def __init__(self, attr, label, v):
        self.attr = attr  # 节点划分的属性索引（-1 表示叶节点）
        self.label = label  # 节点的类别标签（π 表示非叶节点）
        self.attr_v = v  # 父节点划分属性的值（仅对子节点有意义）
        self.children = []  # 子节点列表


### **定义函数验证属性上的值是否相同** 
is_same_on_attr 通常用于决策树构建的特殊情况处理：

检查样本是否无法划分：

在决策树递归过程中，如果样本的所有属性值在指定的属性集合上完全相同，则说明无法通过这些属性进一步划分。
终止递归，生成叶节点：

当所有样本的属性值完全一致时，决策树会选择直接生成叶节点，而不再继续分裂。


In [5]:
def is_same_on_attr(X, attrs):#验证属性上的值是否均相同
    X_a = X[:, attrs]
    target = X_a[0]
    for r in range(X_a.shape[0]):
        row = X_a[r]
        if (row != target).any():
            return False
    return True

### **构建决策树**（函数dicision_tree_init）
递归地构造决策树：

检查是否可以直接生成叶节点。

选择信息增益最大的属性作为划分依据。

对于每个取值生成子节点，并递归调用自身。

In [6]:
def dicision_tree_init(X, Y, attrs, root, purity_cal):
    # 递归基
    if len(set(Y)) == 1:
        root.attr = np.pi
        root.label = Y[0]
        return None

    if len(attrs) == 0 or is_same_on_attr(X, attrs):
        root.attr = np.pi
        # Y 中出现次数最多的label设定为node的label
        root.label = np.argmax(np.bincount(Y))
        return None

    # 计算每个attr的划分收益
    purity_attrs = []
    for i, a in enumerate(attrs):
        p = purity_cal(X, Y, a)
        purity_attrs.append(p)
    #print(purity_attrs)
    chosen_index = purity_attrs.index(max(purity_attrs))
    chosen_attr = attrs[chosen_index]

    root.attr = chosen_attr
    root.label = np.pi

    del attrs[chosen_index]

    x_attr_col = X[:, chosen_attr]
    # 离散数据处理
    for x_v in set(X[:, chosen_attr]):
        n = Node(-1, -1, x_v)
        root.children.append(n)
        # 不可能Dv empty 要是empty压根不会在set里
        # 选出 X[attr] == x_v的行

        index_x_equal_v = np.where(x_attr_col == x_v)
        X_x_equal_v = X[index_x_equal_v]
        Y_x_equal_v = Y[index_x_equal_v]
        dicision_tree_init(X_x_equal_v, Y_x_equal_v, attrs, n, purity_cal)

### **基于构建的决策树进行预测** 决策树预测：dicision_tree_predict



In [7]:
def dicision_tree_predict(x, tree_root):
    if tree_root.label != np.pi:  # 如果是叶节点，返回类别
        return tree_root.label
    for child in tree_root.children:  # 遍历子节点，找到匹配的分支
        if child.attr_v == x[tree_root.attr]:
            return dicision_tree_predict(x, child)
    return None

最后在主函数中进行决策树的构建和基于其进行预测，最后再计算分类准确度：
具体过程是；

加载训练集与测试集。

构建ID3决策树并存储在根节点r。

使用测试集进行预测，并计算分类精度。

In [34]:
if __name__ == '__main__':
    ans = load_txt("Watermelon-train1.csv")
    X_train = ans[:, 1: -1]
    Y_train = ans[:, -1].astype(np.int64)
    test_data = load_txt("Watermelon-test1.csv")
    X_test = test_data[:, 1:-1]
    Y_test = test_data[:, -1].astype(np.int64)
    r = Node(-1, -1, -1)
    attrs = [0, 1, 2, 3]
    dicision_tree_init(X_train, Y_train, attrs, r, gain)
    y_predict = [dicision_tree_predict(x, r) for x in X_test]
    accuracy = sum(1 for y_true, y_pred in zip(Y_test, y_predict) if y_true == y_pred) / len(Y_test)
    print('accuracy:', accuracy)


accuracy: 0.7


分类精度为0.7.

In [ ]:
import numpy as np
import codecs
feature_dict = {"色泽": ["青绿", "乌黑", "浅白"],
                "根蒂": ["蜷缩", "稍蜷", "硬挺"],
                "敲声": ["浊响", "沉闷", "清脆"],
                "纹理": ["清晰", "稍糊", "模糊"]
                }
lable_list = ["否", "是"]
feature_list = ["色泽", "根蒂", "敲声", "纹理"]

def load_txt(path):
    ans = []
    with codecs.open(path, "r", "GBK") as f:
        line = f.readline()  # 读取表头，跳过
        line = f.readline()  # 读取数据第一行
        while line:
            d = line.rstrip("\r\n").split(',')  # 按逗号分割每行数据
            re = []
            re.append(int(d[0]))  # 第0列是编号，作为辅助信息
            re.append(feature_dict.get("色泽").index(d[1]))  # 将“色泽”属性映射为索引
            re.append(feature_dict.get("根蒂").index(d[2]))  # 将“根蒂”属性映射为索引
            re.append(feature_dict.get("敲声").index(d[3]))  # 将“敲声”属性映射为索引
            re.append(feature_dict.get("纹理").index(d[4]))  # 将“纹理”属性映射为索引
            re.append(lable_list.index(d[-1]))  # 最后一列是标签（“是”或“否”），映射为索引
            ans.append(np.array(re))  # 存储为NumPy数组
            line = f.readline()  # 继续读取下一行
    return np.array(ans)

## ***中级要求实现（实现CART算法能够处理连续型特征）***

#### 观察数据集2可知，多了连续型的属性：密度，因此实验需要首先再次观察一下数据集属性组成。

In [55]:
import numpy as np
import codecs

feature_dict = {
    "色泽": ["青绿", "乌黑", "浅白"],
    "根蒂": ["蜷缩", "稍蜷", "硬挺"],
    "敲声": ["浊响", "沉闷", "清脆"],
    "纹理": ["清晰", "稍糊", "模糊"]
}
lable_list = ["否", "是"]#是否是“好瓜,属性名为好瓜”
feature_list = ["色泽", "根蒂", "敲声", "纹理", "密度"]  # 加入新的连续型特征“密度”

def load_txt(path):
    ans = []
    with codecs.open(path, "r", "GBK") as f:
        line = f.readline()  # 读取表头，跳过
        line = f.readline()  # 读取数据第一行
        while line:
            d = line.rstrip("\r\n").split(',')  # 按逗号分割每行数据
            re = []
            re.append(int(d[0]))  # 第0列是编号，作为辅助信息
            re.append(feature_dict.get("色泽").index(d[1]))  # 将“色泽”属性映射为索引
            re.append(feature_dict.get("根蒂").index(d[2]))  # 将“根蒂”属性映射为索引
            re.append(feature_dict.get("敲声").index(d[3]))  # 将“敲声”属性映射为索引
            re.append(feature_dict.get("纹理").index(d[4]))  # 将“纹理”属性映射为索引
            re.append(float(d[5]))  # 第5列是连续型变量“密度”，以浮点数形式读取
            re.append(lable_list.index(d[-1]))  # 最后一列是标签（“是”或“否”），映射为索引
            ans.append(np.array(re))  # 存储为NumPy数组
            line = f.readline()  # 继续读取下一行
    return np.array(ans, dtype=np.float64)  # 指定返回的数组类型为float64

if __name__ == '__main__':
    data = load_txt("Watermelon-train2.csv")
    print(data)

[[ 1.     0.     0.     0.     0.     0.697  1.   ]
 [ 2.     1.     0.     1.     0.     0.774  1.   ]
 [ 3.     1.     0.     0.     0.     0.634  1.   ]
 [ 4.     0.     0.     1.     0.     0.608  1.   ]
 [ 5.     2.     0.     0.     0.     0.556  1.   ]
 [ 6.     0.     1.     0.     0.     0.403  1.   ]
 [ 7.     1.     1.     0.     1.     0.481  1.   ]
 [ 8.     1.     1.     0.     0.     0.437  1.   ]
 [ 9.     1.     1.     1.     1.     0.666  0.   ]
 [10.     0.     2.     2.     0.     0.243  0.   ]
 [11.     2.     2.     2.     2.     0.245  0.   ]
 [12.     2.     0.     0.     2.     0.343  0.   ]
 [13.     0.     1.     0.     1.     0.639  0.   ]
 [14.     2.     1.     1.     1.     0.657  0.   ]
 [15.     1.     1.     0.     0.     0.36   0.   ]
 [16.     2.     0.     0.     2.     0.593  0.   ]
 [17.     0.     0.     1.     1.     0.719  0.   ]]


### 可以看到多了一个属性列为密度，是一个连续变量，后面我们将利用CART算法使用二元切分的方法来处理连续变量密度。

### **数据集的读取**

In [57]:
import numpy as np
import pandas as pd
train2_dataset=pd.read_csv("Watermelon-train2.csv",encoding='GBK')
test2_dataset=pd.read_csv("Watermelon-test2.csv",encoding='GBK')

### **基尼指数的计算实现，以及基于二分方法寻找续型特征；密度的最佳分割点**

#### 注意是/否列属性名为“好瓜”，下面将计算CART算法对应的基尼系数,对于Ck：（𝐷中属于第𝑘类的样本子集，大K是类的个数）公式：

$$Gini(D)=1-\sum_{k=1}^K\left(\frac{|C_k|}{|D|}\right)^2$$

In [72]:
# 计算Gini 指数
def caculate_GINI(data):
    # 计算目标列中每个类别的概率
    class_probs = data['好瓜'].value_counts(normalize=True)
    # 计算 Gini 指数
    gini_index = 1 - sum([p ** 2 for p in class_probs])
    return gini_index

compute_gini_for_split 函数是计算特定特征对于数据集分割后的加权 Gini 指数的函数。它通过遍历特征的所有可能取值，将数据集分割为子集并计算加权 Gini，从而帮助决策树算法选择最优的分割特征。这里使用可以利用 groupby 操作，同时计算加权 Gini 的过程也可以简化。我们可以直接对数据进行分组，并计算每个组的 Gini 值，最后根据每个组的比例计算加权 Gini。

In [87]:
# 计算特定特征的 Gini 分裂指数
def calculate_GINI_for_split(data, feature):
    # 使用groupby分组，并计算每个组的Gini
    weighted_gini = data.groupby(feature).apply(lambda subset: compute_gini_index(subset)).mul(
        data.groupby(feature).size() / len(data)
    ).sum()

    return weighted_gini

对于每个特征：

按照特征值的大小进行排序。

计算相邻两个值之间的中点，作为可能的分割点。

对于每个分割点：

将数据集根据分割点划分为左子集和右子集。

计算左子集和右子集的 Gini 指数。

计算加权 Gini 指数。

选择加权 Gini 指数最小的分割点，作为最佳分割点。

在找到可能的分割点之后，我们将数据集分成两个子集 (左子集和右子集)。每个子集的 Gini 指数可
以单独计算，然后加权平均，以得到整个分割点的加权 Gini 指数。
假设在某个分割点，数据集$D$被划分为两个子集$D_\mathrm{leff}$和$D_\mathrm{right}$,它们的大小分别为$n_\mathrm{left}$和$n_\mathrm{right}$
,且总数据集大小为$n$,则加权 Gini 指数计算公式为：
$$Gini_{{\mathrm{split}}}=\frac{n_{{\mathrm{left}}}}{n}\cdot Gini(D_{{\mathrm{left}}})+\frac{n_{{\mathrm{right}}}}{n}\cdot Gini(D_{{\mathrm{right}}})$$

即：

1. 计算左子集的 Gini 指数$Gini(D_\mathrm{left})$和右子集的 Gini 指数$Gini(D_\mathrm{right})$。
2. 根据左子集和右子集的样本数量，计算加权平均的 Gini 指数。

In [89]:
# 基于 Gini 指数，找到连续型特征的最佳分割点，
def find_best_split_point(data, feature):
    # 获取特征的唯一值并排序
    sorted_values = sorted(data[feature].unique())

    # 计算最佳分割点和最低 Gini
    def calculate_weighted_gini(split_point):
        left_subset = data[data[feature] <= split_point]
        right_subset = data[data[feature] > split_point]

        gini_left = caculate_GINI(left_subset)
        gini_right = caculate_GINI(right_subset)

        weighted_gini = (len(left_subset) / len(data)) * gini_left + (len(right_subset) / len(data)) * gini_right
        return weighted_gini

    # 使用min函数简化计算和比较
    best_split_point, lowest_gini_index = min(
        (( (sorted_values[i] + sorted_values[i + 1]) / 2, calculate_weighted_gini((sorted_values[i] + sorted_values[i + 1]) / 2) )
         for i in range(len(sorted_values) - 1)),
        key=lambda x: x[1]
    )

    return best_split_point, lowest_gini_index
    

### **CART决策树的构建**

**选择最佳分割点**：通过计算所有特征的 Gini 值来选择最佳的分割特征。如果是连续特征，则找到最佳的分割点；如果是离散特征，则直接计算 Gini 指数。

**进行递归地构建子树**：递归地对左右子集或离散特征的子集进行相同的操作。

In [97]:
def build_CART_tree(data, features):
    target = '好瓜'

    # 如果所有目标值相同，返回该目标值（叶节点）
    if data[target].nunique() == 1:
        return data[target].iloc[0]

    # 如果没有剩余的特征用于分裂，返回目标列中最常见的类别（众数）
    if not features:
        return data[target].mode()[0]

    # 初始化最佳属性和 Gini 指数
    best_feature, best_split_point, lowest_gini, is_continuous = None, None, float("inf"), False

    for feature in features:
        if feature == '密度':
            # 连续特征寻找最佳分割点
            split_point, gini_value = find_best_split_point(data, feature)
            if gini_value < lowest_gini:
                best_feature, best_split_point, lowest_gini, is_continuous = feature, split_point, gini_value, True
        else:
            # 离散特征直接计算 Gini 指数
            gini_value = caculate_GINI_for_split(data, feature)
            if gini_value < lowest_gini:
                best_feature, lowest_gini, is_continuous = feature, gini_value, False

    # 构建树的根节点
    cart_tree = {best_feature: {}}

    if is_continuous:
        # 处理连续特征的分割
        left_subset, right_subset = data[data[best_feature] <= best_split_point], data[data[best_feature] > best_split_point]
        cart_tree[best_feature][f'<= {best_split_point}'] = build_CART_tree(left_subset, features)
        cart_tree[best_feature][f'> {best_split_point}'] = build_CART_tree(right_subset, features)
    else:
        # 处理离散特征的分割
        remaining_features = [f for f in features if f != best_feature]
        for value in data[best_feature].unique():
            subset = data[data[best_feature] == value]
            cart_tree[best_feature][value] = build_CART_tree(subset, remaining_features) if not subset.empty else data[target].mode()[0]

    return cart_tree

# 过滤掉目标列和 '编号' 列，获取所有特征
features_cart = [col for col in train2_dataset.columns if col not in ['好瓜', '编号']]


# 构建 CART 决策树
my_CART_tree = build_cart_tree(train2_dataset, features_cart)

#### 这时是我们构建出来的CART决策树应该已经合理地处理密度这个连续特征了。

### **使用CART决策树对测试集dataset2进行预测并计算准确度**

In [116]:
def calculate_cart_accuracy_and_predict(tree, test_data, target_column='好瓜'):
    correct_predictions = 0
    no_branch_predictions = 0  # 记录预测为 None 的样本数

    # 遍历测试数据的每一行，进行预测
    for _, row in test_data.iterrows():
        # 预测一个样本
        def predict_cart(tree, sample):
            # 如果当前节点是叶节点，直接返回预测值
            if not isinstance(tree, dict):
                return tree

            # 遍历当前节点的特征及其子树
            for feature, branches in tree.items():
                if feature in sample:
                    feature_value = sample[feature]

                    # 如果是连续特征的分割，处理 '<=' 和 '>'
                    if isinstance(branches, dict):
                        for branch_condition in branches.keys():
                            # 处理连续特征的分支
                            if branch_condition.startswith('<='): 
                                threshold = float(branch_condition[3:])  # 阈值提取
                                if feature_value <= threshold:
                                    return predict_cart(branches[branch_condition], sample)
                            elif branch_condition.startswith('>'):
                                threshold = float(branch_condition[3:])  
                                if feature_value > threshold:
                                    return predict_cart(branches[branch_condition], sample)

                    # 如果是离散特征的分支，直接递归访问子树
                    if feature_value in branches:
                        return predict_cart(branches[feature_value], sample)

            # 如果没有找到合适的分支，返回 None（无法预测）
            return None

        # 进行预测
        predicted_label = predict_cart(tree, row)
        actual_label = row[target_column]

        # 统计预测正确的样本
        if predicted_label == actual_label:
            correct_predictions += 1
        elif predicted_label is None:
            no_branch_predictions += 1  # 记录没有找到合适分支的情况

    # 计算并返回准确率
    accuracy = correct_predictions / len(test_data)
    return accuracy

# 计算并打印准确率
cart_accuracy = calculate_cart_accuracy_and_predict(my_CART_tree, test2_dataset)
print(f"CART决策树方法的准确度: {cart_accuracy}")

CART决策树方法的准确度: 0.8


#### 可见其准确度较高，我认为原因主要是CART合理地利用好了连续的密度特征信息，从而做出“高明”的分类决策。

## ***高级要求实现（使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝）***

**剪枝**：决策树很容易出现过拟合现象。原因在于学习时完全考虑的是如何提⾼对训练数据的正确分类从⽽构建出过于复杂的决策树。
解决这个问题的方法称为剪枝，即对已生成的树进行简化。具体地，就是从已生成的树上裁剪掉⼀些子树或叶节点，并将其根节点或父节点作为新的叶节点。

决策树的剪枝基本策略有**预剪枝(Pre-Pruning)**和**后剪枝(Post-Pruning)**：

预剪枝：是根据⼀些原则极早的停止树增长，如树的深度达到用户所要的深度、节点中样本个数少于用户指定个数、不纯度指标下降的幅度小于用户指定的幅度等。

后剪枝：是通过在完全生长的树上剪去分枝实现的，通过删除节点的分支来剪去树节点。是在生成决策树之后自底向上的对树中所有的非叶结点进⾏逐一考察 。

剪枝的作用：

避免过拟合：通过减少树的复杂度，剪枝有助于提高模型的泛化能力。

提高模型可解释性：剪枝后的树结构更简洁，减少了冗余的分支。

**这里两个剪枝均对原剪枝树实现后剪枝算法**

### **对基本要求实现的决策树的后剪枝**

**关键函数设计说明**：

post_pruning函数：这是实现后剪枝的主要部分。它递归遍历树的每个节点，并尝试将节点的子树替换为叶节点（即赋予该节点数据集中的多数类标签）。然后，评估剪枝前后的准确度，若剪枝后的准确度不低于原准确度，则保留剪枝后的结构，否则恢复原状。

evaluate_accuracy函数：用于计算剪枝后的树在给定数据集上的准确率。它通过遍历测试集，对每个数据点进行预测并与实际标签比较，从而计算准确率。

dicision_tree_init和dicision_tree_predict：这两个函数用于构建决策树和预测过程，和我之前的实现都一样。

In [112]:
import numpy as np
import codecs

feature_dict = {"色泽": ["青绿", "乌黑", "浅白"],
                "根蒂": ["蜷缩", "稍蜷", "硬挺"],
                "敲声": ["浊响", "沉闷", "清脆"],
                "纹理": ["清晰", "稍糊", "模糊"]
                }
lable_list = ["否", "是"]
feature_list = ["色泽", "根蒂", "敲声", "纹理"]

def load_txt(path):
    ans = []
    with codecs.open(path, "r", "GBK") as f:
        line = f.readline()
        line = f.readline()
        while line:
            d = line.rstrip("\r\n").split(',')
            re = []
            re.append(int(d[0]))
            re.append(feature_dict.get("色泽").index(d[1]))
            re.append(feature_dict.get("根蒂").index(d[2]))
            re.append(feature_dict.get("敲声").index(d[3]))
            re.append(feature_dict.get("纹理").index(d[4]))
            re.append(lable_list.index(d[-1]))
            ans.append(np.array(re))
            line = f.readline()
    return np.array(ans)

class Node:
    def __init__(self, attr, label, v):
        self.attr = attr
        self.label = label
        self.attr_v = v
        self.children = []

def is_same_on_attr(X, attrs):
    X_a = X[:, attrs]
    target = X_a[0]
    for r in range(X_a.shape[0]):
        row = X_a[r]
        if (row != target).any():
            return False
    return True

def ent(D):
    s = 0
    for k in set(D):
        p_k = np.sum(np.where(D == k, 1, 0)) / np.shape(D)[0]
        if p_k == 0:
            continue
        s += p_k * np.log2(p_k)
    return -s

def gain(X, Y, attr):
    x_attr_col = X[:, attr]
    ent_Dv = []
    weight_Dv = []
    for x_v in set(x_attr_col):
        index_x_equal_v = np.where(x_attr_col == x_v)
        y_x_equal_v = Y[index_x_equal_v]
        ent_Dv.append(ent(y_x_equal_v))
        weight_Dv.append(np.shape(y_x_equal_v)[0] / np.shape(Y)[0])
    return ent(Y) - np.sum(np.array(ent_Dv) * np.array(weight_Dv))

def dicision_tree_init(X, Y, attrs, root, purity_cal):
    if len(set(Y)) == 1:
        root.attr = np.pi
        root.label = Y[0]
        return None

    if len(attrs) == 0 or is_same_on_attr(X, attrs):
        root.attr = np.pi
        root.label = np.argmax(np.bincount(Y))
        return None

    purity_attrs = []
    for i, a in enumerate(attrs):
        p = purity_cal(X, Y, a)
        purity_attrs.append(p)

    chosen_index = purity_attrs.index(max(purity_attrs))
    chosen_attr = attrs[chosen_index]
    root.attr = chosen_attr
    root.label = np.pi
    del attrs[chosen_index]

    x_attr_col = X[:, chosen_attr]
    for x_v in set(X[:, chosen_attr]):
        n = Node(-1, -1, x_v)
        root.children.append(n)
        index_x_equal_v = np.where(x_attr_col == x_v)
        X_x_equal_v = X[index_x_equal_v]
        Y_x_equal_v = Y[index_x_equal_v]
        dicision_tree_init(X_x_equal_v, Y_x_equal_v, attrs, n, purity_cal)

def dicision_tree_predict(x, tree_root):
    if tree_root.label != np.pi:
        return tree_root.label

    chose_attr = tree_root.attr
    for child in tree_root.children:
        if child.attr_v == x[chose_attr]:
            return dicision_tree_predict(x, child)
    return None

def post_pruning(X, Y, node):
    if node.label != np.pi:
        return

    for child in node.children:
        post_pruning(X, Y, child)

    original_accuracy = evaluate_accuracy(X, Y, node)
    majority_class = np.argmax(np.bincount(Y))
    node.children = []
    node.label = majority_class
    new_accuracy = evaluate_accuracy(X, Y, node)

    if new_accuracy < original_accuracy:
        node.label = majority_class
        node.children = []

def evaluate_accuracy(X, Y, node):
    predictions = []
    for x in X:
        predictions.append(dicision_tree_predict(x, node))
    correct_predictions = sum(1 for y_true, y_pred in zip(Y, predictions) if y_true == y_pred)
    total_predictions = len(Y)
    return correct_predictions / total_predictions

if __name__ == '__main__':
    ans = load_txt("Watermelon-train1.csv")
    X_train = ans[:, 1: -1]
    Y_train = ans[:, -1].astype(np.int64)
    test_data = load_txt("Watermelon-test1.csv")
    X_test = test_data[:, 1:-1]
    Y_test = test_data[:, -1].astype(np.int64)
    
    r = Node(-1, -1, -1)
    attrs = [0, 1, 2, 3]

    dicision_tree_init(X_train, Y_train, attrs, r, gain)

    post_pruning(X_train, Y_train, r)

    y_predict = []
    for i in range(X_test.shape[0]):
        x = X_test[i]
        y_p = dicision_tree_predict(x, r)
        y_predict.append(y_p)

    correct_predictions = sum(1 for y_true, y_pred in zip(Y_test, y_predict) if y_true == y_pred)
    total_predictions = len(Y_test)
    accuracy = correct_predictions / total_predictions
    print('accuracy after pruning:', accuracy)

accuracy after pruning: 0.5


#### 发现剪枝后的精度为0.5，不如之前的不剪枝的决策树，精度出现了下降

### **对中级要求实现的CART决策树的后剪枝**

#### 主要函数设计说明：

prune_tree: 这是后剪枝的主函数，它递归遍历每个子树，如果在某个节点剪枝后的误差与剪枝前的误差差距较小（小于一个阈值 alpha），就将该节点剪枝为叶节点。

calculate_tree_error: 计算树在验证集上的误差（即预测不准确的比例）。

calculate_tree_error_after_pruning: 计算剪枝后树的误差。剪枝后即用叶节点替代树的所有子树。

get_leaf_value: 用于获取树的叶节点值，即树所有分支的最终预测值。

alpha: 控制剪枝的程度。如果 alpha 为较小值，剪枝会更加激进；较大值时则不会轻易剪枝。

In [121]:
def prune_tree(tree, validation_data, target_column='好瓜', alpha=0.01):
    """
    后剪枝算法，递归删除不重要的分支，以减少过拟合。
    """
    # 如果树是叶子节点，则直接返回
    if not isinstance(tree, dict):
        return tree

    # 遍历树的所有子树
    for feature, branches in tree.items():
        for branch_condition, subtree in branches.items():
            tree[feature][branch_condition] = prune_tree(subtree, validation_data, target_column, alpha)

    # 在当前节点剪枝：计算剪枝前后的误差
    error_before_pruning = calculate_tree_error(tree, validation_data, target_column)
    error_after_pruning = calculate_tree_error_after_pruning(tree, validation_data, target_column)

    # 如果剪枝后的误差不比剪枝前差，进行剪枝
    if error_after_pruning <= error_before_pruning + alpha:
        return get_leaf_value(tree)

    return tree


def calculate_tree_error(tree, data, target_column='好瓜'):
    """
    计算决策树在数据集上的误差
    """
    predictions = data.apply(lambda row: predict_cart(tree, row), axis=1)
    errors = predictions != data[target_column]
    return errors.mean()  # 计算错误率


def calculate_tree_error_after_pruning(tree, data, target_column='好瓜'):
    """
    计算剪枝后树的误差（将所有分支替换为叶节点）
    """
    pruned_tree = get_leaf_value(tree)
    predictions = data.apply(lambda row: predict_cart(pruned_tree, row), axis=1)
    errors = predictions != data[target_column]
    return errors.mean()


def get_leaf_value(tree):
    """
    获取树的叶节点值（即，所有分支的预测结果）
    """
    # 如果是叶子节点，直接返回
    if not isinstance(tree, dict):
        return tree
    for feature, branches in tree.items():
        for branch_condition, subtree in branches.items():
            return get_leaf_value(subtree)


# 构建 CART 决策树
features_cart = [col for col in train2_dataset.columns if col not in ['好瓜', '编号']]

# 构建初始 CART 决策树
my_CART_tree = build_CART_tree(train2_dataset, features_cart)

# 使用验证集进行剪枝
validation_data = train2_dataset  
pruned_tree = prune_tree(my_CART_tree, validation_data, target_column='好瓜', alpha=0.01)

In [122]:
def calculate_cart_accuracy_and_predict(tree, test_data, target_column='好瓜'):
    correct_predictions = 0
    no_branch_predictions = 0  # 记录预测为 None 的样本数

    # 遍历测试数据的每一行，进行预测
    for _, row in test_data.iterrows():
        # 预测一个样本
        def predict_cart(tree, sample):
            # 如果当前节点是叶节点，直接返回预测值
            if not isinstance(tree, dict):
                return tree

            # 遍历当前节点的特征及其子树
            for feature, branches in tree.items():
                if feature in sample:
                    feature_value = sample[feature]

                    # 如果是连续特征的分割，处理 '<=' 和 '>'
                    if isinstance(branches, dict):
                        for branch_condition in branches.keys():
                            # 处理连续特征的分支
                            if branch_condition.startswith('<='): 
                                threshold = float(branch_condition[3:])  # 阈值提取
                                if feature_value <= threshold:
                                    return predict_cart(branches[branch_condition], sample)
                            elif branch_condition.startswith('>'):
                                threshold = float(branch_condition[3:])  
                                if feature_value > threshold:
                                    return predict_cart(branches[branch_condition], sample)

                    # 如果是离散特征的分支，直接递归访问子树
                    if feature_value in branches:
                        return predict_cart(branches[feature_value], sample)

            # 如果没有找到合适的分支，返回 None（无法预测）
            return None

        # 进行预测
        predicted_label = predict_cart(tree, row)
        actual_label = row[target_column]

        # 统计预测正确的样本
        if predicted_label == actual_label:
            correct_predictions += 1
        elif predicted_label is None:
            no_branch_predictions += 1  # 记录没有找到合适分支的情况

    # 计算并返回准确率
    accuracy = correct_predictions / len(test_data)
    return accuracy

# 计算并打印准确率
cart_accuracy = calculate_cart_accuracy_and_predict(pruned_tree, test2_dataset)
print(f"CART决策树方法的准确度: {cart_accuracy}")

CART决策树方法的准确度: 0.8


#### 发现剪枝后的精度仍为0.8，后剪枝没有使得决策树性能更佳。

### **分析精度下降或没有提高的原因**

**1. 剪枝引入的过度简化**

剪枝的目标是通过减少树的复杂性来避免过拟合。然而，在某些情况下，过度剪枝会导致决策树缺乏足够的表达能力，从而降低模型的准确性。

（1）过度剪枝：如果剪枝过早或过度，可能导致模型无法捕捉到训练数据中的复杂模式。尤其是在数据集较复杂或者特征之间的关联性较强的情况下，过度简化可能会减少模型的有效性，从而导致精度下降。

（2）剪枝条件不合理：预剪枝和后剪枝通常都有一些条件，例如节点样本数低于某个阈值、信息增益降低等。如果这些条件设置得不合理，可能会导致模型在树的早期或深度不足的情况下进行剪枝，导致分类性能降低。

**2. 后剪枝操作影响**


（1）后剪枝是通过完全生成树后再进行的操作。尽管这种方法可以通过避免过早停止来防止欠拟合，但它也可能因为过度修剪去除了一些重要的分支，从而导致测试数据集的精度下降。

（2）剪枝步骤：后剪枝通常是自底向上的操作，会从叶子节点逐步回溯，检查每个内部节点是否应该剪去。在进行后剪枝时，如果剪去的分支实际在测试集上具有较高的分类精度，精度可能会显著下降。